In [5]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
from requests.exceptions import HTTPError
import urllib
import urllib.request

from selenium import webdriver
#from reppy.robots import Robots
from urllib.parse import urlparse
import time
import operator

## MtgMintCards

In [17]:
urls = ['http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/the-great-henge',
       'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/oko-thief-of-crowns',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/arcane-signet-brawl-deck',
       'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/questing-beast',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/brazen-borrower',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/knights-charge-brawl-deck',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/shimmer-dragon-brawl-deck',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/steelbane-hydra-brawl-deck',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/murderous-rider',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/once-upon-a-time']
infoMtgmintCard = []

In [18]:
def ExtractMtgMintCards(url):
    
    codigo_fonte = rq.get(url,headers={'content-type': 'text/html'})
    texto = codigo_fonte.text
    soup = BeautifulSoup(texto,'html.parser')
    
    # InfoBox
    root_div = soup.find('div',itemtype="http://schema.org/Product")
    
    #Nome
    Nome = root_div.find('span',itemprop='name').text
    
    #Preço
    preço = '$ ' + root_div.find('span',{'itemprop':'price'}).text
    
    #Nome do Set
    Nome_set = root_div.find('span',content = True).text
    
    #Casting Cost 
    aux_CastingCost = root_div.find('strong',text = 'Casting Cost')
    p_CastingCost = aux_CastingCost.parent
    Lista_Img_castingcost = p_CastingCost.findAll('img',alt=True)
    CastingCost = []
    for image in Lista_Img_castingcost:
        CastingCost.append(image['alt'])    
    
    #Descrição
    aux_descrição = root_div.find('span',{'itemprop':'description'}).text
    descricao = re.sub('[\s+]', ' ', aux_descrição)      
    
    #Raridade 
    aux_Raridade = root_div.find('strong',text = 'Rarity')
    Raridade = aux_Raridade.parent.text.strip('Rarity')
    
    # type
    auxTipo = root_div.find('strong',text='Type')
    Tipo = auxTipo.parent.text.strip('Type')
    
    return infoMtgmintCard.append(dict(zip(['URL','Nome-Carta','Set','Raridade','Casting Cost','Preço','Descrição','Tipo'],
                                           [url,Nome,Nome_set,Raridade,CastingCost,preço,descricao,Tipo])))


In [19]:
for url in urls:
    ExtractMtgMintCards(url)

In [20]:
data = pd.DataFrame(infoMtgmintCard,columns=['URL','Nome-Carta','Set','Raridade','Casting Cost','Preço','Descrição','Tipo'])
data

,URL,Nome-Carta,Set,Raridade,Casting Cost,Preço,Descrição,Tipo
0,http://www.mtgmintcard.com/mtg/singles/eld/eng...,The Great Henge,Throne of Eldraine,Mythic,"[7, Green, Green]",$ 10.99,"This spell costs less to cast, where X is the...",Legendary Artifact
1,http://www.mtgmintcard.com/mtg/singles/eld/eng...,"Oko, Thief of Crowns",Throne of Eldraine,Mythic,"[1, Green, Blue]",$ 37.99,2: Create a Food token. 1: Target artifact o...,Legendary Planeswalker - Oko
2,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Arcane Signet (Brawl Deck),Throne of Eldraine,Common,[2],$ 10.99,: Add one mana of any color in your commander'...,Artifact
3,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Questing Beast,Throne of Eldraine,Mythic,"[2, Green, Green]",$ 35.99,"Vigilance, deathtouch, haste Questing Beast ca...",Legendary Creature - Beast
4,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Brazen Borrower,Throne of Eldraine,Mythic,"[1, Blue, Blue]",$ 15.99,Flash Flying Brazen Borrower can block only cr...,Creature - Faerie Rogu
5,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Knights' Charge (Brawl Deck),Throne of Eldraine,Rare,"[1, White, Black]",$ 4.49,"Whenever a Knight you control attacks, each op...",Enchantment
6,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Shimmer Dragon (Brawl Deck),Throne of Eldraine,Rare,"[4, Blue, Blue]",$ 5.49,Flying As long as you control four or more art...,Creature - Dragon
7,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Steelbane Hydra (Brawl Deck),Throne of Eldraine,Rare,"[X, Green, Green]",$ 4.99,Steelbane Hydra enters the battlefield with X ...,Creature - Turtle Hydra
8,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Murderous Rider,Throne of Eldraine,Rare,"[1, Black, Black]",$ 6.99,"Lifelink When Murderous Rider dies, put it on ...",Creature - Zombie Knight
9,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Once Upon a Time,Throne of Eldraine,Rare,"[1, Green]",$ 19.99,If this spell is the first spell you've cast t...,Instant


## Scryfall

In [171]:
urlsScryfall = ['https://scryfall.com/card/c17/2/balan-wandering-knight','https://scryfall.com/card/ddu/1/ezuri-renegade-leader',
               'https://scryfall.com/card/a25/3/akromas-vengeance','https://scryfall.com/card/txln/2/illusion',
                'https://scryfall.com/card/c17/1/alms-collector','https://scryfall.com/card/hou/158/abandoned-sarcophagus',
               'https://scryfall.com/card/ori/127/abbot-of-keral-keep','https://scryfall.com/card/rna/151/absorb',
               'https://scryfall.com/card/hou/158/abandoned-sarcophagus','https://scryfall.com/card/wwk/20/stoneforge-mystic']
infoScryfall = []

In [172]:
def ExtractScryfall(url):
    codigo_fonte = rq.get(url,headers={'content-type': 'text/html'})
    texto = codigo_fonte.text
    soup = BeautifulSoup(texto,'html.parser')
    
    #Scryfall raiz da carta
    root_div = soup.find('div',class_="inner-flex")
    
    # texto-carta
    carta_texto = root_div.find('div',class_='card-text')
    
    #Casting cost
    auxCastingCost = carta_texto.find('span',class_='card-text-mana-cost')
    if auxCastingCost == None:
        CastingCost = None
    else:
        abbrCastingCost = auxCastingCost.findAll('abbr')
        CastingCost = []
        for abbr in abbrCastingCost:
            simbolo = str(abbr.text)
            simbolo = re.sub('{|}','',simbolo)
            #texto = str(abbr['title'])
            FinalTexto = simbolo #+' '+texto
            CastingCost.append(FinalTexto)
    
    # Descrição
    oracleTexto = carta_texto.find('div',class_="card-text-oracle").text
    flavorTexto = carta_texto.find('div',class_="card-text-flavor")
    if flavorTexto == None:
        descricao = oracleTexto
        descricao = re.sub('[\s+]', ' ', descricao)
    else:
        descricao = oracleTexto + '\n' + flavorTexto.text
        descricao = re.sub('[\s+]', ' ', descricao)    
    
    #Detalhes da edição,preço raridade
    auxDetalhes = root_div.find('div',class_='prints')
    
    # Set
    Set = auxDetalhes.find('span',class_="prints-current-set-name").text
    Set = re.sub('[\s+]', ' ', Set)
    
    # Raridade 
    auxRaridade = auxDetalhes.find('span',class_="prints-current-set-details").text
    auxRaridade = re.sub('[\s+]', '', auxRaridade)
    
    regex =  re.compile('([A-Z]\w+)')    
    infoDetalhes = re.findall(regex,auxRaridade)
    
    Raridade = infoDetalhes[0]
    
    # Idioma
    Idioma = infoDetalhes[1]

    
    # Preço
    Preco = auxDetalhes.find('a',class_="currency-usd").text
        
    # Nome
    # Usar um auxiliar para eliminar tags desnecessarias para retirar o nome.
    aux_carta_texto = root_div.find('div',class_='card-text')
    codetags = aux_carta_texto.findAll('span')
    for codetag in codetags:
        codetag.extract()
    
    Nome = aux_carta_texto.find('h1',class_="card-text-title").text
    Nome = re.sub('[\s+]', ' ', Nome)
    
    #tipo
    auxtipo = aux_carta_texto.find('p',class_="card-text-type-line").text
    Tipo =  re.sub('[\s+]', "", auxtipo)
    
    
    return infoScryfall.append(dict(zip(['URL','Nome-Carta','Set','Raridade','Casting Cost','Preço','Descrição','Tipo','Idioma'],
                                        [url,Nome,Set,Raridade,CastingCost,Preco,descricao,Tipo,Idioma])))


In [173]:
for url in urlsScryfall:
    ExtractScryfall(url)

In [174]:
dataScryfall = pd.DataFrame(infoScryfall,columns=['URL','Nome-Carta','Set','Raridade','Casting Cost','Preço','Descrição',"Tipo","Idioma"])
dataScryfall

,URL,Nome-Carta,Set,Raridade,Casting Cost,Preço,Descrição,Tipo,Idioma
0,https://scryfall.com/card/c17/2/balan-wanderin...,"Balan, Wandering Knight",Commander 2017 (C17),Rare,"[2, W, W]",$8.55,"First strike Balan, Wandering Knight has doub...",LegendaryCreature—CatKnight,English
1,https://scryfall.com/card/ddu/1/ezuri-renegade...,"Ezuri, Renegade Leader",Duel Decks: Elves vs. Inventors (DDU)...,MythicRare,"[1, G, G]",✶ $1.84,{G}: Regenerate another target Elf. {2}{G}{G}...,LegendaryCreature—ElfWarrior,English
2,https://scryfall.com/card/a25/3/akromas-vengeance,Akroma's Vengeance,Masters 25 (A25),Rare,"[4, W, W]",$0.45,"Destroy all artifacts, creatures, and enchant...",Sorcery,English
3,https://scryfall.com/card/txln/2/illusion,Illusion,Ixalan Tokens (TXLN),Common,None,$0.10,When this creature becomes the target of a sp...,TokenCreature—Illusion,English
4,https://scryfall.com/card/c17/1/alms-collector,Alms Collector,Commander 2017 (C17),Rare,"[3, W]",$2.33,Flash If an opponent would draw two or more c...,Creature—CatCleric,English
5,https://scryfall.com/card/hou/158/abandoned-sa...,Abandoned Sarcophagus,Hour of Devastation (HOU),Rare,[3],✶ $0.78,You may cast nonland cards with cycling from ...,Artifact,English
6,https://scryfall.com/card/ori/127/abbot-of-ker...,Abbot of Keral Keep,Magic Origins (ORI),Rare,"[1, R]",✶ $0.98,Prowess (Whenever you cast a noncreature spel...,Creature—HumanMonk,English
7,https://scryfall.com/card/rna/151/absorb,Absorb,Ravnica Allegiance (RNA),Rare,"[W, U, U]",✶ $2.83,Counter target spell. You gain 3 life. “In ...,Instant,English
8,https://scryfall.com/card/hou/158/abandoned-sa...,Abandoned Sarcophagus,Hour of Devastation (HOU),Rare,[3],✶ $0.78,You may cast nonland cards with cycling from ...,Artifact,English
9,https://scryfall.com/card/wwk/20/stoneforge-my...,Stoneforge Mystic,Worldwake (WWK),Rare,"[1, W]",✶ $52.37,When Stoneforge Mystic enters the battlefield...,Creature—KorArtificer,English


## Abugames

In [71]:
urlsAbu = ['https://abugames.com/magic-the-gathering/singles/product-detail/Aggressive-Urge-%252D-FOIL/10th-Edition/English/',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Abundance/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Abundance-%252D-FOIL/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Academy-Researchers/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Academy-Researchers-%252D-FOIL/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Adarkar-Wastes/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Afflict/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Afflict-%252D-FOIL/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Agonizing-Memories/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Flowstone-Salamander/Tempest/English/']
infoAbu = []

In [168]:
def ExtractAbu(url):
    
    # Selenium
    driver = webdriver.Chrome("Driver\\chromedriver78\\chromedriver.exe")
    driver.get(url)
    texto = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit
    
    # Parsing
    soup = BeautifulSoup(texto,'html.parser')
    
    # Tag Informação central
    root_div = soup.find('div',class_="singleCardMain")
    
    # Detalhes
    infoDetalhes = root_div.find('div',class_='row first')
    
    # Nome
    Nome = infoDetalhes.find('h1').text
    
    # Set
    Set = infoDetalhes.find('div',class_="col-md-2 col-sm-2 col-xs-4").text
    
    # Idioma
    auxIdioma = infoDetalhes.find('div',class_="col-md-1 col-sm-1 col-xs-2") 
    
    imgIdioma = auxIdioma.findAll('img',src=True)
    Imagens = []
    for image in imgIdioma:
        Imagens.append(image['src'])
        
    regexIdioma =  re.compile('([A-Z]\w+)')    
    srcImgIdioma = re.search(regexIdioma,Imagens[0])
    
    if srcImgIdioma:
        Idioma = Imagens[0][srcImgIdioma.start():srcImgIdioma.end()]
    else:
        Idioma = None
    
    
    # Raridade
    # o Idioma e a raridade tem a msm classe
    auxRaridade = infoDetalhes.findAll('div',class_="col-md-1 col-sm-1 col-xs-2")
    Raridade = auxRaridade[1].text    
    
    # Descrição 
    originalTexto = root_div.find("div",{"id":"originalTextundefined"}).text
    flavorTexto = root_div.find("div",{"id":"FlavorTextundefined"}).text
    
    Descrição = originalTexto +"\n"+flavorTexto
    
    #Preço
    Preco = root_div.find('div',class_="col-md-1 col-xs-2 ng-star-inserted").text
    
    #TIpo
    Tipo = root_div.find('div',class_="col-md-2 col-sm-2 col-xs-4 ng-star-inserted").text
    
    #Casting cost
    auxCastingcost = infoDetalhes.find('div',class_="col-md-2 col-sm-2 col-xs-3")
    
    tags_i = auxCastingcost.findAll('i')
    
    listaClassTexto = []
    for tag in tags_i:
        texto = tag.attrs.get("class")
        listaClassTexto.append(texto)
   
    CastingCost = []
    for lista in listaClassTexto:
        cost = lista[1]        
        CastingCost.append(re.sub(r'ms-','',cost))
    
    return infoAbu.append(dict(zip(['URL','Nome-Carta','Set','Raridade','Casting Cost','Preço','Descrição','Tipo','Idioma'],
                                   [url,Nome,Set,Raridade,CastingCost,Preco,Descrição,Tipo,Idioma])))



In [169]:
for url in urlsAbu:
    ExtractAbu(url)

[['ms', 'ms-1', 'ms-cost', 'ms-shadow', 'ng-star-inserted'], ['ms', 'ms-g', 'ms-cost', 'ms-shadow', 'ng-star-inserted']]
[['ms', 'ms-2', 'ms-cost', 'ms-shadow', 'ng-star-inserted'], ['ms', 'ms-g', 'ms-cost', 'ms-shadow', 'ng-star-inserted'], ['ms', 'ms-g', 'ms-cost', 'ms-shadow', 'ng-star-inserted']]
[['ms', 'ms-2', 'ms-cost', 'ms-shadow', 'ng-star-inserted'], ['ms', 'ms-g', 'ms-cost', 'ms-shadow', 'ng-star-inserted'], ['ms', 'ms-g', 'ms-cost', 'ms-shadow', 'ng-star-inserted']]
[['ms', 'ms-1', 'ms-cost', 'ms-shadow', 'ng-star-inserted'], ['ms', 'ms-u', 'ms-cost', 'ms-shadow', 'ng-star-inserted'], ['ms', 'ms-u', 'ms-cost', 'ms-shadow', 'ng-star-inserted']]
[['ms', 'ms-1', 'ms-cost', 'ms-shadow', 'ng-star-inserted'], ['ms', 'ms-u', 'ms-cost', 'ms-shadow', 'ng-star-inserted'], ['ms', 'ms-u', 'ms-cost', 'ms-shadow', 'ng-star-inserted']]
[]
[['ms', 'ms-2', 'ms-cost', 'ms-shadow', 'ng-star-inserted'], ['ms', 'ms-b', 'ms-cost', 'ms-shadow', 'ng-star-inserted']]
[['ms', 'ms-2', 'ms-cost', 'ms-

In [170]:
dataAbu = pd.DataFrame(infoAbu,columns=['URL','Nome-Carta','Set','Raridade','Casting Cost','Preço','Descrição','Tipo','Idioma'])
dataAbu

,URL,Nome-Carta,Set,Raridade,Casting Cost,Preço,Descrição,Tipo,Idioma
0,https://abugames.com/magic-the-gathering/singl...,Aggressive Urge - FOIL,10th Edition,Common,g1,$0.89,Target creature gets +1/+1 until end of turn.\...,Instant,English
1,https://abugames.com/magic-the-gathering/singl...,Aggressive Urge - FOIL,10th Edition,Common,"[1, g]",$0.89,Target creature gets +1/+1 until end of turn.\...,Instant,English
2,https://abugames.com/magic-the-gathering/singl...,Abundance,10th Edition,Rare,"[2, g, g]",$2.99,"If you would draw a card, you may instead choo...",Enchantment,English
3,https://abugames.com/magic-the-gathering/singl...,Abundance - FOIL,10th Edition,Rare,"[2, g, g]",$40.79,"If you would draw a card, you may instead choo...",Enchantment,English
4,https://abugames.com/magic-the-gathering/singl...,Academy Researchers,10th Edition,Uncommon,"[1, u, u]",$0.30,"When Academy Researchers comes into play, you ...",Creature - Human Wizard,English
5,https://abugames.com/magic-the-gathering/singl...,Academy Researchers - FOIL,10th Edition,Uncommon,"[1, u, u]",$4.69,"When Academy Researchers comes into play, you ...",Creature - Human Wizard,English
6,https://abugames.com/magic-the-gathering/singl...,Adarkar Wastes,10th Edition,Rare,[],$14.85,: Add to your mana pool.\n: Add or to your ...,Land,English
7,https://abugames.com/magic-the-gathering/singl...,Afflict,10th Edition,Common,"[2, b]",$0.14,Target creature gets -1/-1 until end of turn.\...,Instant,English
8,https://abugames.com/magic-the-gathering/singl...,Afflict - FOIL,10th Edition,Common,"[2, b]",$1.05,Target creature gets -1/-1 until end of turn.\...,Instant,English
9,https://abugames.com/magic-the-gathering/singl...,Agonizing Memories,10th Edition,Uncommon,"[2, b, b]",$0.14,Look at target player's hand and choose two ca...,Sorcery,English
